In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

from tensorflow.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/train.csv
./kaggle/input/test.csv
./kaggle/input/sample_solution.csv


In [10]:
train = pd.read_csv('/opt/ml/mycodes/kaggle/input/train.csv')
test  = pd.read_csv('/opt/ml/mycodes/kaggle/input/test.csv')
sub   = pd.read_csv('/opt/ml/mycodes/kaggle/input/sample_solution.csv')

In [11]:
train['n_missing'] = train.isna().sum(axis=1)
train['n_missing2'] = train.isna().sum(axis=1)*0.5

test['n_missing'] = test.isna().sum(axis=1)
test['n_missing2'] = test.isna().sum(axis=1)*0.5

train['claim'] = train['claim'].astype(str)

features = [col for col in train.columns if col not in ['claim', 'id']]
pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=64,output_distribution='uniform')),
        ('bin', KBinsDiscretizer(n_bins=64, encode='ordinal',strategy='uniform'))
        ])
train[features] = pipe.fit_transform(train[features])
test[features] = pipe.transform(test[features])


In [12]:
model = Sequential([
    Input(train[features].shape[1:]),
    Embedding(input_dim=64, output_dim=4),
    Flatten(),    
    Dense(64,  activation='relu'),
    Dropout(0.5),
    Dense(32,  activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

auc = tf.keras.metrics.AUC(name='aucroc')
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps = 450,
        decay_rate= 0.8)
optimizer = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=[auc]) 

/opt/conda/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
model.fit(x = np.float32(train[features]), y = np.float32(train.claim),
          batch_size = 1024, shuffle = True, epochs = 20)

Epoch 1/20
936/936 [==============================] - 6s 6ms/step - loss: 0.5405 - aucroc: 0.7927
Epoch 2/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5221 - aucroc: 0.8014
Epoch 3/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5213 - aucroc: 0.8022
Epoch 4/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5205 - aucroc: 0.8031
Epoch 5/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5200 - aucroc: 0.8039
Epoch 6/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5193 - aucroc: 0.8049
Epoch 7/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5189 - aucroc: 0.8053
Epoch 8/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5186 - aucroc: 0.8056
Epoch 9/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5185 - aucroc: 0.8057
Epoch 10/20
936/936 [==============================] - 5s 6ms/step - loss: 0.5180 - aucroc: 0.8061
Epoch 11/20
936/936

In [14]:
sub['claim'] = model.predict(np.float32(test[features]))
sub=sub.set_index('id')
sub.to_csv('submission10.csv')